In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pk
import umap
import seaborn
from skimage.measure import label

## Using Filenames.csv from frederic to go from stim names to old file names

In [6]:
import csv,pandas
#the fullVocal Paramater Bank motherload
fileRawData = "/auto/zdrive/lthomas/FullVocBank_2020/vocParamTable.h5"
#fileRawData = os.path.join(acoustic_dir,'vocParamTable_FullVocBank.h5')
vocSelTable = pandas.read_hdf(fileRawData)

vocCategoryTable = pandas.read_hdf('/auto/zdrive/lthomas/FullVocBank_2020/vocParamTable_clusters.h5')
vocSelTable = vocSelTable[vocSelTable.index.isin(vocCategoryTable.index)]
vocSelTable = vocSelTable[vocSelTable['samplerate'] != 0]
vocCategoryTable = vocCategoryTable[vocCategoryTable.index.isin(vocSelTable.index)]


In [14]:
for ind,row in vocSelTable[vocSelTable.filename == 'BlaBla0506_110304-AggC-07'].iterrows():
    print(np.uint8(ind))

128
130
131
135


In [3]:
pdsegs = []
dictFNs = dict()
with open('../data/ephys/wav_mapper.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        fn = row[0].strip('.wav')
        dictFNs[fn] = dict()
        if len(row) == 2:
            wav = row[1].strip()
            sel_rows = vocSelTable[vocSelTable['filename'] == wav]
            if len(sel_rows) == 0:
                print(wav,fn)
                continue
            for ind, srow in sel_rows.iterrows():
                pdsegs.append({'stimname': fn, 'stimstart': 0.0,'stimend': srow.duration,
                               'ind':ind,'wav':wav,'start':srow.inds[0]/srow.samplerate,'end':srow.inds[1]/srow.samplerate})
        else:
            times = [float(x) for x in row[2::2]]
            cumtime=0.0
            for wav,t in zip(row[1::2],times):
                wav = wav.strip()
                sel_rows = vocSelTable[vocSelTable['filename'] == wav]
                if len(sel_rows) == 0:
                    print(wav,fn)
                    break
                    
                for ind, srow in sel_rows.iterrows():
                    pdsegs.append({'stimname': fn, 'stimstart': cumtime,'ind':ind,'wav':wav,
                                   'start':cumtime + srow.inds[0]/srow.samplerate,
                                   'end': cumtime + srow.inds[1]/srow.samplerate,
                                  'stimend':cumtime + srow.duration})

                dur = np.array(sel_rows['duration'])[0]
                cumtime += dur
                cumtime += t

pdsegTable = pandas.DataFrame(pdsegs)
#pdsegTable.to_pickle("../data/ephys/StimSegTable.pkl")
pdsegTable.to_hdf("../data/ephys/StimSegTable_pkl4.h5",'stim_seg',mode='w')
#vocSelTable.to_hdf(store, 'callTable', mode = 'w')

BlaBla0506_110405-Song-15a BlaBla0506_MAF_So_2
LblGre0001_130521-BeggSeq-02 LblGre0001_UCF_Be_0
WhiWhi1415_110405-NestC-04a WhiWhi1415_FAF_Ne_1-3-8
WhiWhi1415_110405-NestC-04b WhiWhi1415_FAF_Ne_2-6-5
BlaLbl8026_110429-DC-20 BlaLbl8026_FAS_DC_8-9-3
BlaLbl8026_110429-DC-11 BlaLbl8026_FAS_DC_2-7-5
BlaLbl8026_110607-WhineC-05 BlaLbl8026_FAS_Wh_5-9-2
BlaLbl8026_110607-WhineC-01 BlaLbl8026_FAS_Wh_1-8-6
BlaLbl8026_110607-WhineC-04 BlaLbl8026_FAS_Wh_4-7-3
BluRas07dd_110407-DC-02 BluRas07dd_FAS_DC_1-3-2
GraLbl0457_110607-AggC-15 GraLbl0457_MAS_Ag_1
GraLbl0457_110428-DC-25 GraLbl0457_MAS_DC_1-4-2
GraLbl0457_110509-DisC-01 GraLbl0457_MAS_Di_0
GraLbl0457_110513-DisC-02 GraLbl0457_MAS_Di_1
GraLbl0457_110509-NestC-03 GraLbl0457_MAS_Ne_3-6-7
GraLbl0457_110606-WhineCop-05 GraLbl0457_MAS_Wh_6-7-3
GraLbl0457_110606-WhineCop-02 GraLbl0457_MAS_Wh_4-9-1
GraLbl0457_110606-WhineCop-04 GraLbl0457_MAS_Wh_5-2-8
BlaBla0506_110405-Song-15a BlaBla0506_MAF_So_3
GraGra0201_110615-TetC-01 GraGra0201_FAF_Te_5-6-2
GraG

In [3]:
pdsegTable = pandas.read_pickle("../data/ephys/StimSegTable.pkl")

ValueError: unsupported pickle protocol: 5

### Setting the input file name (this could be in a GUI)

In [6]:
## For Seweissen data: 
# summary_pkl_path= "/auto/zdrive/lthomas/summarySel.pkl
# output_dir = "/auto/zdrive/lthomas/goodpkls_plots/""
# goodpkls_path = "/auto/zdrive/lthomas/goodpkls/"
# For Julie Data:
summary_pkl_path = "/auto/zdrive/lthomas/julie/summarySelJulie.pkl"
goodpkls_path = "/auto/zdrive/lthomas/julie/goodpkls/"
output_plot_dir = "/auto/zdrive/lthomas/julie/figs/goodpkls_plots/"

summary = pk.load(open(summary_pkl_path,'rb'))
# pAud and decide threshold
print(summary[(summary['zAud'] > .5)&(summary['selInd'] > .1)].shape)
#summary[summary.site == 'L1500R1500'].unit[6]
site='L1500R1500'
bird = 'YelBlu6903F'
print(summary[(summary['site']==site) & (summary.bird == bird)].shape)

(208, 12)
(81, 12)


## Convert to Kevin's dataframe type

Structure:
'call_type', 'electrode', 'exposure', 'isi_histogram',
       'isi_violation_pct', 'n_trials', 'relation', 'self', 'site', 'snr',
       'spike_times', 'stim_duration', 'unit', 'unit_id', 'reward_class',
       'stimulus_file', 'subject', 'date'

In [9]:
sites = list(set(zip(summary.site,summary.bird)))
for site in sites:
    site = list(site)
    print("Site: %s \t Count %d"%(site,((summary.site==site[0]) & (summary.bird==site[1])).sum()))

Site: ['L2000R1600', 'BlaBro09xxF'] 	 Count 53
Site: ['L750R1100', 'GreBlu9508M'] 	 Count 74
Site: ['L1400R1400', 'LblBlu2028M'] 	 Count 56
Site: ['L1800R2100', 'GreBlu9508M'] 	 Count 86
Site: ['L1500R1450', 'WhiWhi4522M'] 	 Count 55
Site: ['L1100R1450', 'GreBlu9508M'] 	 Count 82
Site: ['L2450R2350', 'WhiWhi4522M'] 	 Count 67
Site: ['L1500R1500', 'BlaBro09xxF'] 	 Count 47
Site: ['L1500R1500', 'YelBlu6903F'] 	 Count 81
Site: ['L500R400', 'YelBlu6903F'] 	 Count 51
Site: ['L1000R900', 'YelBlu6903F'] 	 Count 83
Site: ['L1100R1275', 'WhiWhi4522M'] 	 Count 46
Site: ['L1200R1200', 'YelBlu6903F'] 	 Count 83
Site: ['L2100R2050', 'WhiWhi4522M'] 	 Count 71
Site: ['L1250R1650', 'GreBlu9508M'] 	 Count 69
Site: ['L1500R1900', 'GreBlu9508M'] 	 Count 72
Site: ['L1900R1800', 'WhiWhi4522M'] 	 Count 71
Site: ['L2500R2300', 'BlaBro09xxF'] 	 Count 60


In [10]:
# First lets try for one site
#dUnitData = {'call_type': [], 'n_trials': [],
#       'spike_times': [], 'stim_duration': [], 'unit': [], 'unit_id': [],
#       'stimulus_file':[], 'wav':[], 
#       'subject':[], 'site': [], 'electrode': [], 'snr': []}
#site='L1500R1500'
#bird = 'BlaBro09xxF'
pad = .5#s
for site,bird in sites[1:]:
    dUnitData = {'call_type': [], 'n_trials': [],
       'spike_times': [], 'stim_duration': [], 'unit': [], 'unit_id': [],
       'stimulus_file':[], 'wav':[], 
       'subject':[], 'site': [], 'electrode': [], 'snr': []}
    for ind,row in summary[(summary['site']==site) & (summary.bird == bird)].iterrows():
        unit_fn = row['unit'].split('/')[-1]
        path = "%sbirds/%s/%s/%s"%(goodpkls_path,row['bird'],row['site'],unit_fn)
        print(path)

        fileIn = open(path, 'rb')
        unitInfo = pk.load(fileIn)
        dfAbsTime = pk.load(fileIn)
        dfRelTime = pk.load(fileIn)
        fileIn.close()
        electrode = int(unitInfo['Electrode'])
        site_name = unitInfo['Site']
        bird_name = unitInfo['Bird']
        snr = unitInfo['SpikeSNR']
        for relind, relrow in dfRelTime.iterrows():
            sfn = relrow.file[:-4]#[:-10]
            # get segments associated with this stimulus
            segs = pdsegTable[pdsegTable['stimname'] == sfn]
            if len(segs)==0:
                # find start and end of wav
                wf = relrow.stimWav
                nonzeros = np.convolve(wf!=0,np.array([1,1,1]))[1:-1]
                segments,n_segs = label(nonzeros != 0,return_num=True)

                for i,seg in enumerate(range(n_segs)):
                    #first unit wide data
                    dUnitData['subject'].append(bird_name)
                    dUnitData['site'].append(site_name)
                    dUnitData['electrode'].append(electrode)
                    dUnitData['snr'].append(snr)
                    uid = unit_fn[:-4]
                    dUnitData['unit_id'].append(uid)
                    dUnitData['unit'].append(int(uid[-1]))

                    dUnitData['call_type'].append(relrow['call_type'])
                    dUnitData['n_trials'].append(relrow['nTrials'])

                    this_seg = np.where(segments == seg)[0]
                    start = relrow.tStim[this_seg[0]]
                    end = relrow.tStim[this_seg[-1]]
                    sts = [ st[(st > start - pad) & (st <= end + pad)] - start 
                                           for st in relrow.spikeTimes]
                    dUnitData['spike_times'].append(sts)
                    dUnitData['stim_duration'].append(end - start)
                    dUnitData['stimulus_file'].append(sfn)
                    dUnitData['wav'].append('Missing_%s_%s'%(sfn,i))
            else:
                for segind, segrow in segs.iterrows():
                    #first unit wide data
                    dUnitData['subject'].append(bird_name)
                    dUnitData['site'].append(site_name)
                    dUnitData['electrode'].append(electrode)
                    dUnitData['snr'].append(snr)
                    uid = unit_fn[:-4]
                    dUnitData['unit_id'].append(uid)
                    dUnitData['unit'].append(int(uid[-1]))

                    dUnitData['call_type'].append(relrow['call_type'])
                    dUnitData['n_trials'].append(relrow['nTrials'])
                    sts = [ st[(st > segrow.start - pad) & (st <= segrow.end + pad)] - segrow.start 
                                           for st in relrow.spikeTimes]

                    dUnitData['spike_times'].append(sts)
                    dUnitData['stim_duration'].append(segrow.end - segrow.start)
                    dUnitData['stimulus_file'].append(segrow['stimname'])
                    dUnitData['wav'].append(segrow['wav'])
    dUnitData = pandas.DataFrame(dUnitData)
    #dUnitData.to_pickle("../data/ephys/UnitData_%s_%s.pkl"%(bird,site))
    dUnitData.to_hdf("../data/ephys/UnitData_%s_%s.h5"%(bird,site),'unit_data')

/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e15_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e24_s1_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e27_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e23_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e20_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e16_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e13_s0_ss3.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e9_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e10_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508M/L750R1100/Site1_L750R1100_e5_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/GreBlu9508

/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['call_type', 'site', 'spike_times', 'stimulus_file', 'subject', 'unit_id', 'wav']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e10_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e14_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e8_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e3_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e27_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e22_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e10_s0_ss1.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e10_s0_ss3.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e11_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodpkls/birds/LblBlu2028M/L1400R1400/Site1_L1400R1400_e14_s0_ss2.pkl
/auto/zdrive/lthomas/julie/goodp

In [11]:
print("DONE")

DONE


## Now to get population vector from unitData

For each stimulus, I will take all of the units during that presentation, and make a n-dimensional array (number of units). I will make an entry for each 400 ms window (shifted by 10 ms) which will be a nx1 array of the average number of spikes over that 400 ms.

In [12]:
#dUnitData = pd.read_pickle("../data/ephys/UnitData_%s_%s.pkl"%(bird,site))
dUnitData = pd.read_hdf("../data/ephys/UnitData_%s_%s.h5"%(bird,site),'unit_data')

In [13]:
sites = list(set(zip(summary.site,summary.bird)))
for site in sites:
    site = list(site)
    print("Site: %s \t Count %d"%(site,((summary.site==site[0]) & (summary.bird==site[1])).sum()))

Site: ['L2000R1600', 'BlaBro09xxF'] 	 Count 53
Site: ['L750R1100', 'GreBlu9508M'] 	 Count 74
Site: ['L1400R1400', 'LblBlu2028M'] 	 Count 56
Site: ['L1800R2100', 'GreBlu9508M'] 	 Count 86
Site: ['L1500R1450', 'WhiWhi4522M'] 	 Count 55
Site: ['L1100R1450', 'GreBlu9508M'] 	 Count 82
Site: ['L2450R2350', 'WhiWhi4522M'] 	 Count 67
Site: ['L1500R1500', 'BlaBro09xxF'] 	 Count 47
Site: ['L1500R1500', 'YelBlu6903F'] 	 Count 81
Site: ['L500R400', 'YelBlu6903F'] 	 Count 51
Site: ['L1000R900', 'YelBlu6903F'] 	 Count 83
Site: ['L1100R1275', 'WhiWhi4522M'] 	 Count 46
Site: ['L1200R1200', 'YelBlu6903F'] 	 Count 83
Site: ['L2100R2050', 'WhiWhi4522M'] 	 Count 71
Site: ['L1250R1650', 'GreBlu9508M'] 	 Count 69
Site: ['L1500R1900', 'GreBlu9508M'] 	 Count 72
Site: ['L1900R1800', 'WhiWhi4522M'] 	 Count 71
Site: ['L2500R2300', 'BlaBro09xxF'] 	 Count 60


In [ ]:


#subject = 'YelBlu6903F'
#site='L1500R1500'
def create_popVec(site,subject,df_unit_data):
    df_unit_data = pd.read_hdf("../data/ephys/UnitData_%s_%s.h5"%(subject,site),'unit_data')
    pad = .5#s from before
    window_size = .05 # 50 ms windows
    window_shift = .01 # 10 ms shift
    aud_delay = .01 # consider a 10ms delay before we call something the played label
    dUnitData_subj_site = df_unit_data[(df_unit_data.subject == subject) & (df_unit_data.site == site)]
    units = np.unique(dUnitData_subj_site.unit_id)
    stims = np.unique(dUnitData_subj_site.wav)
    call_types = ['None', 'Ag', 'Be', 'DC', 'Di', 'LT', 'Ne', 'Ri', 'So', 'Te', 'Th']

    dfPopulationVectors = {
        "pop_vectors": [],
        "pv_labels" : [],
        "source_labels" : [],
        "rel_time": [],
        "window": [],
    }
    print(len(stims))
    for stim in stims:
        dUnitData_subj_site_stim = dUnitData_subj_site[dUnitData_subj_site.wav == stim]
        # all should have same trials 
        ntrials = dUnitData_subj_site_stim.iloc[0].n_trials
        # and same duration
        dur = dUnitData_subj_site_stim.iloc[0].stim_duration + pad * 2
        n_windows = int((dur - window_size)//window_shift)

        pv = np.zeros((int(ntrials*n_windows),len(units)))
        n_pre = int((pad + aud_delay - window_size) // window_shift)
        n_post = int(( pad - aud_delay - window_size) // window_shift)
        n_stim = n_windows - n_pre - n_post
        lab = ['None'] * n_pre + [ dUnitData_subj_site_stim.iloc[0].call_type ] * n_stim + ['None'] * n_post
        source_lab = [ stim ] * n_windows
        dfPopulationVectors['pv_labels'].extend(lab * ntrials)
        time = list(np.arange(n_windows)*window_shift - pad)
        dfPopulationVectors['rel_time'].extend(time * ntrials)
        dfPopulationVectors['window'].extend(ntrials*list(range(-n_pre,n_windows-n_pre)))
        for ti in range(ntrials):
            source_lab = [ "%s_Trial%d"%(stim,ti) ] * n_windows
            dfPopulationVectors['source_labels'].extend(source_lab)
            for wi in range(n_windows):
                for ui,unit in enumerate(units):
                    unit_row = dUnitData_subj_site_stim[dUnitData_subj_site_stim.unit_id == unit]
                    sts = unit_row.iloc[0].spike_times[ti]
                    win_start = window_shift * wi
                    win_end = win_start + window_size
                    pv[ti*n_windows + wi, ui] = ((sts > win_start) & (sts < win_end)).sum() / window_size
        dfPopulationVectors['pop_vectors'].extend(pv)
    
    dfPopulationVectors = pd.DataFrame(dfPopulationVectors)
    #dfPopulationVectors.to_pickle("../data/ephys/%s_%s_Population_vectors_%d.pkl"%(subject,site,int(window_size*1000)))
    dfPopulationVectors.to_hdf("../data/ephys/Population_Vectors/%s_%s_Population_vectors_%d.h5"%(subject,site,int(window_size*1000)),'pop_vec')
    return dfPopulationVectors
#dfpv = create_popVec(site,bird,dUnitData)
for site,subject in sites[1:]:
    print(site,subject)
    create_popVec(site,subject,dUnitData)

L750R1100 GreBlu9508M
284


/auto/fhome/lthomas/.virtualenvs/py3_env/lib/python3.5/site-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['pop_vectors', 'pv_labels', 'source_labels']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


L1400R1400 LblBlu2028M
528
L1800R2100 GreBlu9508M
284


In [22]:

dfPopulationVectors = pd.DataFrame(dfPopulationVectors)
dfPopulationVectors.to_pickle("../data/ephys/Population_vectors.pkl")

In [48]:
dUnitData_subj_site = dUnitData[(dUnitData.subject == bird) & (dUnitData.site == site)]
units = np.unique(dUnitData_subj_site.unit_id)
dfpv['rel_time'] = []
stims = np.unique(dUnitData_subj_site.wav)
for stim in stims:
    dUnitData_subj_site_stim = dUnitData_subj_site[dUnitData_subj_site.wav == stim]
    # all should have same trials 
    ntrials = dUnitData_subj_site_stim.iloc[0].n_trials
    # and same duration
    dur = dUnitData_subj_site_stim.iloc[0].stim_duration + pad * 2
    n_windows = int((dur - window_size)//window_shift)
    time = list(np.arange(n_windows)*window_shift - pad)
    #time = list(np.arange(-pad,n_windows*window_shift - pad,window_shift))
    dfpv['rel_time'].extend(time * ntrials)

In [52]:

dfpv = pd.DataFrame(dfpv)
#dfpv.to_pickle("../data/ephys/%s_%s_Population_vectors_%d.pkl"%(bird,site,int(window_size*1000)))
dfpv.to_h5("../data/ephys/%s_%s_Population_vectors_%d.h5"%(bird,site,int(window_size*1000)),'pop_vec')


In [20]:
278000/2

139000.0